In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ---------------- Load Excel Data ----------------
file_path = "S7 track record.xlsx"
# Excel file has multi-row headers (rows 7–8), so we set header=[6, 7] (0-based)
df = pd.read_excel(file_path, header=[6, 7])

# Flatten MultiIndex column names for easier access
df.columns = [f"{a}_{b}" if "Unnamed" not in str(b) else str(a) for a, b in df.columns]

# Rename key columns for clarity
df = df.rename(columns={
    "Register No": "RegNo",
    "Name of the student": "Name"
})

# Drop any rows without a registration number
df = df.dropna(subset=["RegNo"])
# Ensure registration numbers are strings and stripped of extra spaces
df["RegNo"] = df["RegNo"].astype(str).str.strip()
df = df.reset_index(drop=True)

# ---------------- Define Semesters ----------------
semesters = ["S1", "S2", "S3", "S4", "S5", "S6"]

# ---------------- Create Output Directory ----------------
output_dir = "track_record"
os.makedirs(output_dir, exist_ok=True)

# ---------------- Generate Per-Student Progress Reports ----------------
for idx, row in df.iterrows():
    regno = row["RegNo"]
    name = str(row["Name"]).strip().replace("/", "-").replace("\\", "-")  # Clean filename

    # Extract back paper counts for each semester
    backpapers = []
    for sem in semesters:
        back_col = [col for col in df.columns if col.startswith(f"{sem}_Back")]
        if back_col:
            back_count = row[back_col[0]]
        else:
            back_count = 0
        backpapers.append(back_count)

    # Extract total back papers if available; fallback to sum
    total_cols = [col for col in df.columns if "Total back papers" in col]
    total_back = row[total_cols].values[0] if len(total_cols) > 0 else sum(backpapers)

    # ---------------- Plot Semester Back Papers ----------------
    plt.figure(figsize=(6, 4))
    plt.bar(semesters, backpapers, color="teal")
    plt.title(f"Progress Report: {name} ({regno})")
    plt.xlabel("Semester")
    plt.ylabel("Back Papers")
    plt.ylim(0, max(backpapers) + 2)

    # Display total back papers on the chart
    center_x = (len(semesters) - 1) / 2
    plt.text(center_x, max(backpapers) + 1,
             f"Total Back Papers: {total_back}",
             ha='center', fontsize=12, color="red", weight="bold")

    # Save chart as PNG image per student
    file_name = f"{regno}_{name}.png"
    file_path = os.path.join(output_dir, file_name)
    plt.savefig(file_path, bbox_inches="tight")
    plt.close()

print(f"Reports saved in folder: {output_dir}")


Reports saved in folder: track_record
